In [125]:
import urllib2     
import numpy as np  
import findspark     
import pymysql
findspark.init()  
from pyspark import SparkContext, SparkFiles, SQLContext

In [126]:
if not 'sc' in locals():  
    sc = SparkContext()  

if not 'sqlContext' in locals():
    sqlContext = SQLContext(sc)
#wordsDF = sqlContext.createDataFrame([('cat',), ('elephant',), ('rat',), ('rat',), ('cat', )], ['word'])   
#wordCountsDF = wordsDF.groupBy("word").count()   
#wordCountsDF.show() 

## k-means & steaming 

In [127]:
import numpy as np
def cluster_centroids(data, clusters, k=None):
    """Return centroids of clusters in data.

    data is an array of observations with shape (A, B, ...).

    clusters is an array of integers of shape (A,) giving the index
    (from 0 to k-1) of the cluster to which each observation belongs.
    The clusters must all be non-empty.

    k is the number of clusters. If omitted, it is deduced from the
    values in the clusters array.

    The result is an array of shape (k, B, ...) containing the
    centroid of each cluster.

    >>> data = np.array([[12, 10, 87],
    ...                  [ 2, 12, 33],
    ...                  [68, 31, 32],
    ...                  [88, 13, 66],
    ...                  [79, 40, 89],
    ...                  [ 1, 77, 12]])
    >>> cluster_centroids(data, np.array([1, 1, 2, 2, 0, 1]))
    array([[ 79.,  40.,  89.],
           [  5.,  33.,  44.],
           [ 78.,  22.,  49.]])

    """
    if k is None:
        k = np.max(clusters) + 1
    result = np.empty(shape=(k,) + data.shape[1:])
    for i in range(k):
        np.mean(data[clusters == i], axis=0, out=result[i])
    return result

import scipy.spatial

def kmeans(data, k=None, centroids=None, steps=20):
    """Divide the observations in data into clusters using the k-means
    algorithm, and return an array of integers assigning each data
    point to one of the clusters.

    centroids, if supplied, must be an array giving the initial
    position of the centroids of each cluster.

    If centroids is omitted, the number k gives the number of clusters
    and the initial positions of the centroids are selected randomly
    from the data.

    The k-means algorithm adjusts the centroids iteratively for the
    given number of steps, or until no further progress can be made.

    >>> data = np.array([[12, 10, 87],
    ...                  [ 2, 12, 33],
    ...                  [68, 31, 32],
    ...                  [88, 13, 66],
    ...                  [79, 40, 89],
    ...                  [ 1, 77, 12]])
    >>> np.random.seed(73)
    >>> kmeans(data, k=3)
    array([1, 1, 2, 2, 0, 1])

    """
    if centroids is not None and k is not None:
        assert(k == len(centroids))
    elif centroids is not None:
        k = len(centroids)
    elif k is not None:
        # Forgy initialization method: choose k data points randomly.
        centroids = data[np.random.choice(np.arange(len(data)), k, False)]
    else:
        raise RuntimeError("Need a value for k or centroids.")

    for _ in range(max(steps, 1)):
        # Squared distances between each point and each centroid.
        sqdists = scipy.spatial.distance.cdist(centroids, data, 'sqeuclidean')

        # Index of the closest centroid to each data point.
        clusters = np.argmin(sqdists, axis=0)

        new_centroids = cluster_centroids(data, clusters, k)
        if np.array_equal(new_centroids, centroids):
            break

        centroids = new_centroids

    return clusters

In [128]:
def clr(tmp):
    tmp = tmp.replace(',),','')
    tmp = tmp.replace(',','')
    tmp = tmp.replace(',)','')
    tmp = tmp.replace('(','')
    tmp = tmp.replace(')','')
    tmp = tmp.replace('\"','')
    tmp = tmp.replace('?','')
    #tmp = tmp.replace('/',' ')
    tmp = tmp.replace(".\'",'')
    tmp = tmp.replace("u'",'')
    tmp = tmp.replace("'s",'')
    tmp = tmp.replace(".",'')
    tmp = tmp.replace(":",'')
    tmp = tmp.lower()
    tmp = tmp.replace('-','')
#    for i in range(100):
 #       tmp = tmp.replace(str(i),'')
    return tmp

In [129]:
IT_word =['1g or analog', 'abpd', 'absorption chillers', 'access media gateways', 'access method', 'accountable care organization', 'accredited standards committee', 'acr', 'acris', 'active matrix organic light-emitting diodes', 'activity-based costing', 'activity-based management', 'ad ops', 'addressable tv advertising', 'adsorption chillers', 'advanced clinical research information systems', 'advanced distribution management systems', 'advanced distribution protection and restoration devices', 'advanced fraud detection and analysis technologies', 'advanced planning and scheduling', 'advanced research projects agency network', 'advanced server energy monitoring tools', 'advanced shipment notice', 'advanced technology group', 'advanced threat detection', 'advertisement action', 'advocacy marketing', 'aerial and digital imagery', 'agent collaboration tools', 'agent portal solutions', 'agile marketing project management', 'agile neorad', 'aio pcs', 'algorithmic business', 'all-in-one', 'alliance and relationship management', 'alm paas', 'also lean sensei', 'alternative card networks', 'alternative delivery models', 'ambient and glanceable displays', 'american standard code for information interchange', 'amo', 'answer marketplace', 'applicant tracking systems', 'application delivery controller', 'application infrastructure suite', 'application life cycle management platform as a service', 'application modernization services', 'application obfuscation', 'application platform as a service', 'application release automation', 'application software services', 'channel', 'technology and industry', 'application-specific integrated circuit', 'application-specific standard product', 'applications portfolio analysis', 'aps', 'arad', 'architected rapid application development', 'model-driven development', 'assemble to order', 'asset performance management', 'atd appliances', 'audio messaging interchange specification', 'audio mining/speech analytics', 'auto-tiering', 'automated document factory', 'automated payment tracking and reconciliation services', 'automated system operations', 'automated vehicle locating', 'automatic storage tiering', 'autonomous system number', 'average inventory', 'average selling price', 'average speed of answer', 'b2b gateway software', 'back-end', 'backup/recovery software', 'bandwidth as currency', 'bank tiers', 'bapi', 'bar code marketing', 'basic input/output system', 'basic trading area', 'battery-powered radio frequency identification', 'behavior learning engines', 'behavioral/gestural analytics', 'best-in-class', 'best-of-breed', 'bi platforms', 'bi services', 'bill of lading', 'bill-of-materials', 'binary large object', 'binary runtime environment for wireless', 'binary synchronous communications', 'bioinformatics and chemoinformatics', 'biometric atms', 'bipv', 'blanking interval', 'ble', 'blu-ray', 'bluetooth 3.0', 'bluetooth 4.0', 'bpm pure-play', 'bpus', 'brand service company', 'branded content management', 'bre', 'bring your own device', 'bt3.0', 'bt4.0', 'build-operate-transfer', 'building-integrated photovoltaics', 'business application programming interface', 'business capability modeling', 'business continuity management planning', 'business continuity manager', 'business pattern recognition', 'business process analysis tools', 'business process as a service', 'business process re-engineering', 'business process utilities', 'business rule engines', 'business service provider', 'c-band', 'c-commerce', 'c/im software', 'cable service provider', 'cagr', 'call detail recording', 'cam', 'capable-to-promise', 'capacity requirements planning', 'car connectivity consortium', 'cargo portals', 'carrier network infrastructure', 'casbs', 'case', 'case management solutions', 'case-based reasoning', 'catalog content management', 'ccas', 'cd-rw', 'cellular multi-processing', 'cellular to wi-fi authentication', 'cert', 'certified network professional', 'channel service unit', 'charge-coupled device', 'chip design starts', 'circuit grade', 'circuit-switched data', 'citizen developer', 'cko', 'claims analytics', 'claims management solutions', 'class of service', 'class-based queuing', 'classless inter-domain routing', 'clear to send', 'click-through', 'click-through rate', 'clicks and bricks', 'client appliance', 'client computing hardware services', 'client management tools', 'climate-driven forecasting', 'clinical information modeling initiative', 'clinical kiosks', 'clocking', 'closed user group', 'closed-loop material requirements planning', 'closed-loop mrp', 'closed-loop performance management', 'cloud access security brokers', 'cloud advertising', 'cloud communications service provider', 'cloud email', 'cloud encryption gateways', 'cloud printing services', 'cloud security gateways', 'cloud sourcing deals anatomy', 'cloud-based grid computing', 'cluster controller', 'cmip over llc', 'cmip over tcp/ip', 'cmns', 'co-creation', 'codd\xe2\x80\x99s rule zero', 'collective competency', 'college of healthcare information management executives', 'column-store database management system', 'com', 'comanagement processes', 'combined heat and power', 'commerce everywhere', 'commerce platform servers', 'commercial telematics', 'committed access rate', 'common internet file system', 'common messaging calls', 'common open software environment', 'common public radio interface', 'communications as a service', 'communications oriented production information and control system', 'community of practice', 'compact disc-rewritable', 'compare operational readiness evaluation', 'complementary metal-oxide semiconductor', 'complex-event processing', 'compliance unit', 'component object library', 'component-based development', 'composite content applications', 'comprehensive production architecture', 'computational-fluid-dynamic', 'computer output to laserdisc', 'computer output to microfilm or microfiche', 'computer-aided design and drafting', 'computer-aided engineering', 'computer-aided manufacturing', 'computer-aided patient-entered medical history', 'computer-aided software engineering', 'computer-based patient record', 'computer-based physician order entry', 'computer-brain interface', 'computer-integrated manufacturing', 'computer-telephony integration', 'computerized maintenance management system', 'computing appliance', 'computing platform products', 'concurrent backup', 'concurrent database restore', 'connected game handhelds', 'connected imaging devices', 'connected portable media players', 'connected portable navigation devices', 'connection-oriented service', 'consolidated service desk', 'constant linear velocity', 'consultative selling', 'consumer broadband connections', 'consumer broadband services', 'consumer digital rights management', 'consumer internet services', 'consumer nas', 'consumer network attached storage', 'consumer telematics', 'consumer voice access lines', 'consumer voice services', 'consumer web mashups', 'consumer-generated media', 'contact center infrastructure', 'contact center system', 'contact center workforce optimization', 'contact database', 'contactless payments', 'content and applications service provider', 'content marketing platforms', 'content-aware data loss prevention', 'context delivery architecture', 'context management engine', 'context-aware computing', 'context-aware security', 'context-enriched content', 'context-enriched services', 'contextual presence', 'continuous controls monitoring', 'contract life cycle management', 'control objectives for information and related technology', 'converged network services', 'convergent charging', 'cooling management systems', 'core banking system', 'corporate and major enterprise', 'corporate license agreement', 'corporate property management software', 'corporate trade exchange', 'corporate website activity', 'cost per click', 'cost per gross add', 'cost to serve', 'cpe business dsl router with embedded dsl modem', 'cpe external stand-alone modem', 'cpe internal modem', 'cpemh', 'cpm suites', 'cpq application suites', 'crisis/incident management', 'critical path method or critical path management', 'critical to quality', 'critical-path scheduling', 'crm printing', 'crm publishing', 'cross-certification', 'cross-compiler', 'cross-docking', 'cross-modulation', 'ctp systems', 'cumulative lead time', 'custom key set', 'customer communications management', 'customer engagement center', 'customer gateways', 'customer information file', 'customer information management', 'customer information management and application', 'customer self-service and support', 'customer service and support', 'customized network management', 'cyber incident response plan', 'cyber incident response team', 'cyber library', 'daisy-chaining', 'dapp', 'data analysis and provider profiling', 'data center outsourcing', 'data center storage encryption', 'data dependency mapping', 'data loss protection', 'data management and integration', 'data ops', 'data quality software as a service', 'data-driven marketing', 'database audit and protection', 'ddvn', 'deal sweet spot analysis', 'defects per million opportunities', 'defects per opportunity', 'defects per unit', 'measure', 'analyze', 'improve', 'control', 'demand pattern analysis', 'demand signal repository', 'demand-driven value network', 'deming pdca cycle', 'design for six sigma', 'design for x', 'design-to-demand', 'desktop outsourcing', 'development and integration services', 'device-embedded biometric authentication', 'diagnostic analytics', 'dif', 'digital ad operations platforms', 'digital agencies', 'digital audio broadcasting plus', 'digital audiotape', 'digital commerce experience', 'digital cross-connect', 'digital dial tone', 'digital loopback', 'digital marketing hub', 'digital pathology scanners', 'digital thick wallet solutions', 'digital thin wallet solutions', 'digital transformation consulting', 'direct chip cooling', 'direct to home', 'direct-access storage device', 'directorate general of telecommunications', 'dirty protocols', 'disciplined multisourcing', 'discretionary security controls', 'dish/stirling engine', 'proprietary', 'distributed tape for backup and archiving', 'distribution requirements planning', 'docomo java', 'document management hardware services', 'document-enabled vertical application', 'doja', 'dpmo', 'buffer', 'rope', 'dsl/cable-sharing residential/small-office gateway/router', 'dual-attached station', 'dual-band', 'dual-band network', 'dual-view displays', 'dvcs', 'dxc', 'dye sublimation', 'dye-sensitized solar cells', 'dynamic application security testing', 'dynamic data masking', 'dynamic web application tools', 'e-book readers', 'e-business', 'e-coupons', 'e-crm', 'e-discovery software', 'e-forms', 'e-learning', 'e-mobility', 'e-otd', 'e-prescribing', 'e-procurement', 'e-readers', 'e-signature', 'e-waste', 'ea assurance', 'eao', 'economizers', 'eh&s applications', 'electro mobility', 'electronic bill presentment and payment', 'embedded cellular data', 'embedded software and electronics design', 'emerging data protection schemes', 'ems', 'endpoint protection platform', 'energy trading and risk management', 'enhanced data rates for global evolution', 'enhanced e-books', 'enhanced network delivery', 'ens', 'ensemble interactions', 'enterprise and public network', 'enterprise application outsourcing', 'enterprise information archiving', 'enterprise mobile communications gateway', 'enterprise nervous system', 'enterprise security intelligence', 'enterprise technology architecture', 'enterprise unified communications infrastructure', 'enterprise-class', 'enterprise-grade', 'entity resolution and analysis', 'entry-level smartphone', 'environmental health and safety', 'er&a', 'erasable paper printing systems', 'fault and log management', 'event-driven architecture', 'event-triggered marketing', 'external services provider', 'external social software', 'extreme low-energy servers', 'f-commerce', 'fabric-based computing', 'fabric-based infrastructure', 'facebook commerce', 'falm', 'fast-packet switching', 'fault detection and isolation', 'feature smartphone', 'federated application life cycle management', 'fiber to the home', 'fiber to the premises', 'fibre channel over ethernet', 'field sales/inside sales', 'field service management software', 'financial analytical applications', 'financial footprint', 'finished branded product', 'firecall', 'first generation', 'first pass yield', 'fixed-line carrier', 'fixed-mobile convergence', 'flexible media and conferencing switching', 'footfall analysis', 'foreign/global trade compliance', 'formula/recipe management', 'fpy', 'freedom of mobile multimedia access', 'frequency-hopping multiple access', 'gain sharing', 'gan', 'gemba', 'gemba walk', 'geospatial mashups', 'geothermal cooling for data centers', 'geothermal power generation', 'global regulatory management', 'green money', 'grid computing without using public cloud computers', 'gross new connections', 'han', 'hand-off', 'handheld device markup language', 'haptics in automotive', 'hardware and software maintenance services', 'hardware maintenance and support services', 'hcm application services', 'hcm in social software', 'hcpcs', 'hdml', 'head-mounted displays', 'head-up displays', 'health insurance portability and accountability act', 'health plan employer data and information set', 'healthcare infomediary', 'healthcare procedural classification system', 'hedis', 'help desk management', 'hierarchical storage management and archive software', 'high performance workplace', 'high tj systems', 'high-concentration photovoltaics', 'high-definition', 'high-performance computing', 'high-speed color inkjet printing', 'high-temperature superconductivity', 'his', 'historian software', 'home location register', 'home-area network', 'hospital information system or healthcare information system', 'hosted pc virtualization software', 'hosted virtual desktops', 'hr disintermediation', 'hr shared-service-center tools', 'http 2.0', 'hundred call seconds', 'hybrid storage gateway appliance', 'hybrid thinking', 'hybrid unified communications and collaboration', 'hydrogen fuel cell vehicles', 'hyperprotocol', 'iam as a service', 'iamaas', 'icd-10 adoption technology', 'idars', 'ide', 'identity and access intelligence', 'identity and access management', 'identity-aware networks', 'identity-proofing services', 'iefm', 'ilpt', 'iltc', 'ims', 'imt-2000', 'imt-a', 'in erlangs', 'in-rack cooling', 'in-row cooling', 'incentive compensation management', 'industrialized it services', 'infocentric', 'infocomm development authority of singapore', 'infonomics', 'information capabilities framework', 'infrared data association', 'infrastructure as a service', 'infrastructure utility services', 'institute of electrical and electronics engineers', 'instruction set virtualization', 'instructor-led practical training', 'instructor-led training in the classroom', 'integrated applications environment', 'integrated definition methodology', 'integrated digital enhanced network', 'integrated document archive and retrieval system', 'integrated e-form management', 'integrated marketing management', 'integrated publishing architecture', 'integration appliances', 'integration as a service', 'integration platform as a service', 'intercompany multimodal unified communications', 'internal rate of return', 'international mobile equipment identifier', 'international mobile subscriber identity', 'international mobile telecommunications-2000', 'international organization for standardization', 'internet document security', 'internet message access protocol', 'internet of things', 'internet sales outlet', 'interoperable storage encryption', 'ip centrex platforms', 'ip extension line', 'ip multimedia subsystem session control layer', 'ip seat license', 'ip-enabled pbx', 'ip-enabled pbx/kts ip extension line', 'ip-pbx', 'ip-service control points', 'ipaas', 'it asset management', 'it chargeback', 'it consulting', 'it consulting services', 'it industrialization', 'it infrastructure utility', 'it management services', 'it operations management software', 'it service desk', 'it service support management tools', 'it services sourcing', 'it services strategic sourcing', 'it/ot alignment', 'it/ot integration', 'itam', 'itom', 'itsd', 'itssm tools', 'ius', 'java platform ad tools', 'just in time', 'k-map', 'ka-band', 'kaikaku', 'kerfless wafering', 'key process input variable', 'key process output variable', 'knowledge content owners', 'knowledge content specialists', 'knowledge work management', 'kohai', 'korea communications commission', 'kpiv', 'kpov', 'ku-band', 'kurtz-above band', 'kurtz-under band', 'kyoto wheel', 'l-band', 'lagging and leading key performance indicators', 'lan bridging', 'language-oriented development environments', 'laser disk', 'legacy application or system', 'lifetime clinical record', 'light-emitting diode', 'light-emitting polymer', 'lights-out recovery operations management', 'limited disclosure technology', 'line dot matrix', 'line hit', 'line impact dot matrix', 'line inkjet', 'line load control', 'line of business', 'line of code', 'link redundancy level', 'liquid crystal on silicon', 'lithium battery backup', 'local modifications memory', 'local number portability', 'local-area network', 'location-aware applications', 'location-aware technology', 'location-based advertising', 'location-based marketing', 'location-based services', 'longitudinal redundancy check', 'loosely coupled multiprocessing', 'low-cost it', 'lowest common denominator', 'loyads', 'lte-a', 'm-business', 'm-commerce', 'machine-to-machine', 'magic quadrant', 'magnetic-ink character recognition', 'magneto-optic', 'mini and midrange', 'mainstream notebook', 'managed network services', 'managed service provider', 'management services organization', 'marketing automation system', 'marketing content management', 'marketing database system', 'marketing encyclopedia system', 'marketing mix and product portfolio optimization', 'marketing mix modeling', 'marketing mix optimization', 'marketing performance management', 'marketing resource management', 'marketing service providers', 'marketscope', 'massive open online course', 'master content management', 'materials management information system', 'mdm services', 'media-embedded merchandising', 'mem', 'message numbering', 'message warehouse', 'messaging application programming interface', 'messaging device', 'messaging hypertext markup language', 'metadata and data modeling tools', 'micro-marketplace', 'microbrowser', 'microinverters', 'microsoft certified professional', 'million instructions per second', 'minutes of use', 'mobile and wireless infrastructure software platforms', 'mobile centrex', 'mobile cloud synchronization', 'mobile deep packet inspection', 'mobile device management services', 'mobile high-definition link', 'mobile pc odms', 'mobile transformers', 'mobile virtual network enabler', 'mobile voice over internet protocol', 'mobile wireless local loop', 'mobile-network operator', 'mobile/wireless portal', 'mobility managed services', 'model-driven architecture', 'model-driven packaged applications', 'molten salt energy storage', 'mous', 'adds and changes', 'moving picture experts group', 'mpeg-4 advanced video coding', 'multichannel campaign management', 'multichannel feedback management', 'multichannel loyalty', 'multifunction product', 'multimodal tms', 'multipurpose internet messaging extensions', 'mvne', 'n-channel metal-oxide semiconductor', 'nanography', 'nanomaterial supercapacitors', 'native code compiler for java', 'natural-language processing', 'natural-language understanding', 'nccj', 'nccm tools', 'ncop', 'necessary non-value-adding', 'net new connections', 'net present value', 'net-liberated organization', 'network access point', 'network and system management', 'network code of practice', 'network configuration and change management', 'network dynamic functionality', 'network fault monitoring tools', 'network on chip', 'network performance monitoring tools', 'network performance tuning/configuring facilities', 'network security silicon', 'network virtual terminal', 'network-attached storage', 'neural net or neural network', 'neurobusiness', 'next-generation firewalls', 'next-generation network', 'next-generation telematics protocol', 'nexus of forces', 'ngfws', 'ngtp', 'nmt-f', 'no-email initiatives', 'non-cable consumer fixed broadband', 'non-value-adding', 'nonwireline cellular carrier', 'object late binding', 'object transaction monitor', 'object-based middleware', 'object-based technologies', 'object-oriented analysis and design', 'object-oriented database management system', 'object-oriented programming', 'object-oriented system', 'object-oriented technology', 'off the shelf', 'offshore programming', 'oms', 'on-device monitoring', 'on-device portal', 'online complex processing', 'open compute project', 'open mobile alliance', 'operational resource management', 'operational technology', 'opnfv', 'opportunity management system', 'organic light-emitting diode', 'over the air', 'own design manufacturer', 'pace-layered application strategy', 'packet assembler/disassembler', 'packet-switched network', 'partial document encryption', 'partner customer support software', 'pattern-based strategy', 'pc virtual software appliance', 'pclm', 'peak transaction rate', 'peer-to-peer', 'performance-driven culture', 'persona management', 'personal access communication services', 'personal access system', 'phase alternate line', 'phase-shift keying', 'phmt', 'photonic crystal displays', 'physical resource and infrastructure management tools', 'physician contracting organization', 'physician hospital organization', 'plain old xml', 'platform as a service', 'platform-independent', 'pnds', 'point of presence', 'point of sale', 'point of service', 'point to point', 'poke-yoke', 'portable flash media for content distribution', 'portable storage device control', 'portal fabric', 'portal-enabling middleware', 'portals and user interaction tools', 'pos', 'postmodern erp', 'postpaid connection', 'power adaptive algorithms', 'predictive behavior analysis', 'predictive modeling solutions', 'prerelational dbms', 'print markets and management hardware', 'printed semiconductors', 'problem management', 'process analytical technology', 'line', 'procure-to-pay solution', 'product analytics', 'product cost and life cycle management', 'product portfolio and program management', 'product support services', 'professional services administration', 'protocol-transparent', 'ptt poc', 'public-key cryptography', 'public-key infrastructure', 'publish and subscribe', 'pull printing', 'pulse carrier', 'push-to-talk over cellular', 'quad-band', 'quality of service', 'quality of storage service', 'quantum dot displays', 'r-uim', 'rack mount', 'radio packet assembler/disassembler', 'radio pad', 'radio-frequency identification', 'rapid architected application development', 'redaction tools', 'redundant array of independent disks', 'availability and serviceability', 'removable modules', 'removable user identity module', 'repeatable solutions', 'requirements definition and management', 'resource requirements planning', 'retail digital signage', 'retail execution and monitoring', 'retirements', 'return on competitiveness', 'return on equity', 'return on information technology', 'return on investment', 'reusable analog intellectual property', 'risk management and compliance consulting services', 'risk-adjusted value management', 'roit', 'rotary heat exchanger', 'round-trip time', 'runtime application self-protection', 'rural service area', 's-band', 's-http', 'sales analytics', 'sales configuration systems', 'sales enablement', 'sales force automation \xe2\x80\x94 direct sales', 'sales to end users', 'sales-focused partner relationship management', 'sam', 'sam tools', 'satellite broadcasting operator', 'satellite communications operator', 'satellite navigation solutions', 'sc-fdma', 'scalable vector graphics interface', 'sciv', 'screen popping', 'scriptless testing', 'search-based data discovery tools', 'secondary station', 'secret-key cryptography', 'secure hypertext transport protocol', 'secure web gateway', 'security information and event management', 'security information and event management software', 'seiban', 'selective sourcing', 'self-describing messages', 'self-relocating program', 'self-service analytics', 'self-service business intelligence', 'self-test and fault isolation', 'sell side', 'semi-mobile wireless', 'senpai', 'sensor and rfid-based inventory and asset management business process application software', 'sensor and rfid-based inventory and asset management infrastructure software', 'serial dot matrix', 'serial inkjet', 'server virtualization infrastructure', 'server virtualization management', 'serverless printing', 'service management agreement', 'service parts planning', 'service provider routers and switches', 'service switching point', 'service-level agreement', 'service-level management', 'service-level objective', 'service-oriented architecture', 'serving area', 'serving gprs support node', 'seven wastes', 'sff-lr', 'shadow/mirror databases', 'shared services or shared services center', 'shielded pair', 'shortest path first', 'shrink-wrapped', 'signal-to noise ratio', 'signaling gateways/ip-stps', 'silicon anode batteries', 'simple api for xml', 'simplex circuit', 'simulation routines', 'single channel per carrier', 'single in-line memory module', 'single large expensive disk', 'single minute exchange of die', 'single sign-on', 'single-carrier frequency division multiple access', 'single-image mode', 'single-mode fiber', 'single-pass device', 'skills-based management', 'skinless servers', 'sma', 'small and midsize business', 'small office/home office', 'small-form-factor', 'legacy-reduced', 'small-to-midsize enterprise', 'smart mfp', 'smart pills', 'smfp', 'soa governance technologies', 'social software standards', 'socialcasting', 'society for worldwide interbank financial telecommunication', 'softswitch architecture', 'softswitches', 'software as a service', 'software change and configuration management', 'software failure detection and recovery', 'software oversight committee', 'software support services', 'software-defined networks', 'software-defined radio', 'solid-state appliances', 'sometimes ymck or \xe2\x80\x9cprocess color printing\xe2\x80\x9d', 'spaghetti chart', 'specialized mobile radio', 'tracking', 'evaluation', 'production', 'spectrum harmonization', 'speech circuit', 'ssem', 'stalking horses', 'standard performance evaluation corporation', 'statement of work', 'statistical quality control/statistical process control', 'statistical time-division multiplexing', 'stock-keeping unit', 'storage area network', 'storage subsystem hardware services', 'storage support services', 'store-and-forward manager', 'store-and-forward voice', 'strategic information office', 'iteration', 'testing', 'enablement', 'scripting', 'populating and channel outputs', 'sub-11ghz proprietary bwa', 'sub-voice-grade channel', 'objective', 'assessment and plan', 'subnet mask', 'subnet number', 'subscriber acquisition cost', 'subscriber identity module card', 'super video graphics array', 'super wi-fi', 'supervisory control and data acquisition', 'supply chain architecture life cycle', 'supply chain inventory visibility', 'supply chain segmentation', 'svg interface', 'switch-based storage virtualization', 'switched service network', 'synchronization markup language', 'synchronized bills of materials', 'system and server evaluation model', 'system-managed storage', 'systematized nomenclature of medicine', 'systems administration management tools', 'technical license management', 'technology-enabled relationship management', 'telecom analytics', 'telecom equipment support services', 'telephony-centric uc approach', 'tera-architectures', 'terabit-per-second transport', 'test data generator', 'tethered remote access', 'thin provisioning', 'three-schema architecture', 'through-silicon vias', 'total access communications system', 'total connections', 'total cost of ownership', 'total iptv subscribers', 'total service revenue', 'touch-sensitive', 'track and trace', 'transmission-level point', 'trunk media gateways', 'ultra-high-speed broadband internet', 'ultramobiles', 'unified communications and collaboration', 'unified communications products', 'user experience platforms', 'uxp', 'value-added network', 'value-added reseller', 'value-added service', 'value-adding', 'valueops', 'vbscript', 'vbx', 'vendor risk management', 'vendor-neutral', 'versatile authentication server and service', 'very high-speed digital subscriber lines', 'very large-scale integration', 'very small aperture terminal', 'video on demand', 'virtual matrix organization', 'virtual server facility', 'virtual tape library', 'visual basic extension', 'visual basic for applications', 'vno', 'voice application servers', 'voice endpoint', 'voice of the customer', 'voice over internet protocol', 'voice over wireless lan', 'control and applications', 'voice-enabled residential/small-office gateway/router with embedded dsl modem', 'vsca', 'wan', 'wan performance monitor', 'wan replacement', 'wap browser', 'wap identity module', 'waste walk', 'wavelength division multiplexing passive optical network', 'wdm-pon', 'weak or strong signal', 'web 2.0', 'web access management', 'web authorization management', 'web conferencing and shared work spaces/team collaboration', 'web e-mail', 'web integration servers', 'web-enabled', 'web-oriented architecture', 'website experience analytics', 'weca', 'wholesale carrier', 'wi-fi', 'wi-fi alliance', 'wi-fi mesh', 'wi-fi protected access', 'wi-fi protected access 2', 'wibree', 'wide-area network', 'wide-area network optimization controllers', 'wihd', 'wimedia alliance', 'wired for management', 'wireless application gateway', 'wireless datagram protocol', 'wireless ethernet compatibility alliance', 'wireless intrusion prevention system', 'wireless local-area network', 'work breakdown structure', 'worldwide interoperability for microwave access', 'xdsl iad', 'zero-latency enterprise', 'enterprise edition', 'micro edition', 'multiple data', 'abc', 'abm', 'access point', 'aco', 'act i', 'active data dictionary', 'active directory', 'activex', 'activity stream', 'ad services', 'adapters', 'adaptive learning', 'adaptive multirate', 'adas', 'adc', 'additive manufacturing', 'adf', 'adms', 'advanced analytics', 'advanced driver assistance systems', 'advanced message queuing protocol', 'advanced metering infrastructure', 'advanced mobile phone service', 'advanced technology', 'advanced web services', 'affective computing', 'ahp', 'aim', 'ais', 'amd', 'american medical informatics association', 'american national standards institute', 'amg', 'ami', 'amia', 'amis', 'amoled', 'amps', 'amqp', 'amr', 'analytic applications', 'analytical hierarchy process', 'analytics', 'ansi', 'aol instant messenger', 'apa', 'apaas', 'apm', 'applet', 'appliances', 'application', 'application architecture', 'application control', 'application development', 'application infrastructure', 'application integration', 'application management', 'application management outsourcing', 'application performance monitoring', 'application program', 'application server', 'application service provider', 'application sharing', 'applications outsourcing', 'ara', 'architected', 'architecture', 'arpanet', 'artificial intelligence', 'asa', 'asc', 'ascii', 'asic', 'asn', 'aso', 'asp', 'assp', 'asynchronous transfer mode', 'asynchronous transmission', 'atg', 'atm', 'ats', 'attenuation', 'augmented reality', 'authentication service', 'authentication technologies', 'authorization', 'automated backup', 'automated business process discovery', 'automated teller machine', 'automated testing', 'automatic content recognition', 'automatic restart', 'autonomous system', 'autonomous vehicles', 'availability', 'avatar', 'avc', 'avl', 'b2b web services', 'backbone', 'backbone network', 'backbone router', 'background task', 'backhaul', 'backplane', 'backup server', 'balanced scorecard', 'bam', 'band', 'bandwidth', 'base station', 'base station controller', 'base transceiver station', 'baseband', 'batch processing', 'baud', 'bcm', 'bcmp', 'bcp', 'benchmarking', 'best practice', 'beta testing', 'bia', 'bicc', 'bid', 'big data', 'bigdata', 'bimodal', 'binary code', 'binary digit', 'binary table', 'bioacoustic sensing', 'biochips', 'bioinformatics', 'biomass', 'biometric authentication', 'biometric characteristic', 'bios', 'bit', 'blade servers', 'blob', 'block error rate', 'blog', 'blogging platforms', 'bluetooth', 'bluetooth low energy', 'bom', 'bookmark', 'bottleneck', 'bpa', 'bpaas', 'bpf', 'bpm', 'bpm standards', 'bpmss', 'bpo', 'bpr', 'brand extension', 'brew', 'broadband', 'broadband wireless access', 'broadcast', 'broadcast storm', 'brownout', 'browser', 'bsc', 'bsp', 'bta', 'bts', 'buffer', 'bug', 'bundling', 'burst', 'business activity monitoring', 'business analytics', 'business continuity planning', 'business impact analysis', 'business intelligence', 'business intelligence competency center', 'business process', 'business process automation', 'business process fusion', 'business process management', 'business process management suites', 'business process modeling', 'business process outsourcing', 'bwa', 'byod', 'byte', 'byte code', 'c++', 'caas', 'cac', 'cache', 'caching server', 'cadd', 'cae', 'call center', 'call processing', 'caller id', 'campaign management', 'campaign management system', 'capacity utilization', 'captive centers', 'car', 'care delivery organization', 'carrier frequency', 'carrier system', 'cascading faults', 'category management', 'cbd', 'cbq', 'cbr', 'ccc', 'ccd', 'ccm', 'ccow', 'ccs', 'cdma', 'cdma2000', 'cdns', 'cdo', 'cdp', 'cdr', 'cec', 'cell', 'cell site', 'cell splitting', 'cellular radio', 'cem', 'central processing unit', 'certificate revocation list', 'certification authority', 'certification practice statement', 'certified information systems security professional', 'cfd analysis', 'cgi', 'cgm', 'change management', 'channel', 'channel analytics', 'channel bank', 'channel capacity', 'channel integration', 'check digit', 'checksum', 'chief information officer', 'chief knowledge officer', 'chief technology officer', 'chime', 'chp', 'cidr', 'cif', 'cifs', 'cim', 'cima', 'cimi', 'cio', 'cir', 'circuit', 'circuit board', 'circuit switching', 'cirp', 'cirt', 'cisc', 'cissp', 'cla', 'cladding', 'clamshell', 'class', 'cli', 'clickstream analysis', 'client', 'client/server', 'clinical context object workgroup', 'clinical data repository', 'clinical resource management', 'clipping', 'clm', 'clock', 'cloud application development', 'cloud computing', 'cloud engineering', 'cloud file sharing', 'cloud management platforms', 'cloud service elasticity', 'cloud services brokerage', 'cloudbursting', 'clustered system', 'clustering', 'clv', 'cmc', 'cme', 'cmms', 'cmol', 'cmos', 'cmos radio frequency', 'cmot', 'cmp', 'cms', 'cmv', 'cmyk', 'cni', 'cnp', 'cns', 'coaxial cable', 'cobit', 'coda', 'code division multiple access', 'codec', 'coder/decoder', 'cognitive radio', 'col', 'cold', 'collaborative commerce', 'collaborative product commerce', 'collective', 'commerce experiences', 'commercial', 'commercial parallel processing', 'committed information rate', 'common access card', 'common gateway interface', 'common information model', 'common language infrastructure', 'common object request broker architecture', 'communications port', 'communications service provider', 'communities', 'community', 'community cloud', 'compandor', 'competency center', 'competitive intelligence', 'complex instruction set computer', 'compliance', 'component', 'component object model', 'composite application', 'composition', 'compound annual growth rate', 'computer emergency response team', 'concentrator', 'concurrent engineering', 'concurrent use', 'conditioning', 'conductor', 'configuration auditing', 'configuration management', 'configure', 'connect time', 'connected devices', 'connected home', 'connection', 'connection admission control', 'connection mode network service', 'connectionless', 'connectionless service', 'consent management', 'console', 'constant bit rate', 'consumer telepresence', 'consumerization', 'contact center', 'contactless card', 'content', 'content aggregation', 'content aggregator', 'content analytics', 'content awareness', 'content delivery networks', 'content management', 'content management systems', 'content marketing', 'content optimization', 'content provider', 'content targeting', 'contention', 'context', 'contingent workforce', 'continuity check', 'continuous data protection', 'continuous improvement', 'continuous integration', 'continuous moves', 'continuous operations', 'continuous process improvement', 'continuous production', 'continuous quality improvement', 'control character', 'control charts', 'control code', 'controlled medical vocabulary', 'conversational media', 'cookie', 'cooperative processing', 'coordination mechanics', 'copics', 'corba', 'core', 'core storage management', 'corporate performance management', 'cos', 'cose', 'cost optimization', 'country code', 'course management', 'cpa', 'cpc', 'cpe', 'cpga', 'cpi', 'cpm', 'cpms', 'cpoe', 'cpp', 'cpr', 'cpr system', 'cpri', 'cps', 'cpt', 'cpu', 'cqi', 'crc', 'critical ratio', 'critical success factor', 'crl', 'crm', 'crm analytics', 'crowdsourcing', 'crp', 'crud', 'csb', 'csd', 'csf', 'csp', 'css', 'csu', 'cti', 'cto', 'ctq', 'ctr', 'ctx', 'cug', 'current loop', 'current procedural terminology', 'customer analytics', 'customer experience', 'customer experience management', 'customer premises equipment', 'customer relationship management', 'cycle time', 'cyclic redundancy check', 'dab', 'dab+', 'daemon', 'dam', 'dap', 'dark data', 'dark fiber', 'darpa', 'das', 'dasd', 'dash7', 'dashboards', 'dast', 'dat', 'data broker', 'data center', 'data center bridging', 'data center infrastructure management', 'data deduplication', 'data definition language', 'data integration', 'data interchange format', 'data lake', 'data mining', 'data monetization', 'data profiling', 'data quality tools', 'data replication', 'data scientist', 'data synchronization', 'data warehouse', 'data wiping', 'database activity monitoring', 'database appliances', 'database design', 'database encryption', 'database management system', 'dbms', 'dbms management', 'dbs', 'dcb', 'dcim', 'dco', 'ddbms', 'ddl', 'ddm', 'deep learning', 'defense advanced research projects agency', 'define', 'demand forecasting', 'demand response', 'deployment', 'descriptive analytics', 'design thinking', 'desktop virtualization', 'deva', 'device resource management', 'devops', 'dfss', 'dfx', 'dgt', 'digital', 'digital audio broadcasting', 'digital business', 'digital commerce', 'digital customer', 'digital divide', 'digital forensics', 'digital marketer', 'digital marketing', 'digital marketing strategy', 'digital modem', 'digital multimedia broadcasting', 'digital network', 'digital rights management', 'digital signature', 'digital subscriber line', 'digital switching', 'digital wallet', 'digital watermarking', 'digital workplace', 'digitalization', 'digitization', 'digitize', 'direct broadcast satellite', 'direct channel', 'directed speech recognition', 'directory access protocol', 'directory services', 'disaster recovery', 'disaster recovery planning', 'discrete manufacturing', 'discussion database', 'discussion forum', 'distributed antenna system', 'distributed application platform', 'distributed computing', 'distributed data management', 'distributed database', 'distributed database management system', 'distributed function', 'distributed generation', 'distributed platforms', 'distributed power generation', 'distributed request', 'distributed resource management', 'distributed system management', 'distributed version control system', 'dlp', 'dmaic', 'dmb', 'dmi', 'dna logic', 'dock scheduling', 'document management', 'domain name', 'downlink', 'download', 'downtime', 'dpi', 'dpo', 'dpu', 'drm', 'drp', 'drum', 'dsl', 'dsm', 'dsr', 'dsscs', 'dtc', 'dth', 'dual mode', 'dumb terminal', 'dynamic address translation', 'dynamic business process management', 'dynamic content', 'dynamic routing', 'ea tools', 'eam', 'eap', 'earth station', 'ebpp', 'ecm', 'eda', 'edc', 'edge', 'efm', 'ehr', 'eia', 'eim', 'elastic multitenancy', 'electronic coupons', 'electronic customer relationship management', 'electronic data capture', 'electronic forms', 'electronic health record', 'electronic paper', 'electronic prescribing', 'electronic signature', 'electronic waste', 'electrovibration', 'embedded analytics', 'emcg', 'emergent structures', 'emm', 'emotion detection', 'emss', 'encapsulation', 'encryption', 'energy management systems', 'enhanced messaging service', 'enhanced observed time difference', 'ensemble programming', 'enterprise application software', 'enterprise applications', 'enterprise architecture', 'enterprise asset management', 'enterprise content management', 'enterprise context', 'enterprise fraud management', 'enterprise information architecture', 'enterprise information management', 'enterprise metadata management', 'enterprise performance management', 'enterprise portal', 'enterprise resource planning', 'enterprise risk management', 'enterprise social software', 'enterprise solutions', 'entitlement management', 'epm', 'epp', 'erm applications', 'erp', 'esi', 'esp', 'eta', 'ethernet', 'ethernet services', 'etrm', 'etsi', 'european telecommunications standards institute', 'event', 'expert system', 'expertise location', 'extensible authentication protocol', 'extensible business reporting language', 'extensible markup language', 'extensible stylesheet language', 'extensible stylesheet language transformations', 'extranet', 'eye tracking', 'fabric computing', 'facilities management', 'factory scheduling', 'failure modes effects analysis', 'fbc', 'fbi', 'fcc', 'fcoe', 'fdd', 'fddi', 'federal communications commission', 'federated identity management', 'federated search', 'femtocell', 'femtocells', 'fhma', 'fiber distributed data interface', 'fiber optics', 'field service management', 'file server', 'file transfer protocol', 'finite loading', 'firewall', 'fixed wireless', 'flat panel display', 'flexible manufacturing system', 'fmc', 'fmea', 'fms', 'folksonomies', 'folksonomy', 'foma', 'forensic software', 'fpd', 'framework', 'framing', 'fraud detection', 'free cooling', 'frequency', 'frequency band', 'frequency division duplex', 'frequency modulation', 'frequency reuse', 'ftp', 'ftth', 'fttp', 'fuel cell', 'function point', 'functional programming languages', 'fuzzy logic', 'galileo', 'gallium nitride', 'gamification', 'gateway', 'gateway server', 'gaze control', 'generation scheduling', 'generic access network', 'geo', 'geofencing', 'geographic information system', 'geostationary satellite', 'geosynchronous orbit', 'gesture control', 'gesture recognition', 'gis', 'global delivery', 'global delivery model', 'global positioning system', 'glonass', 'gps', 'granularity', 'graphical user interface', 'gray market', 'gray scale', 'green belt', 'grid computing', 'ground segment', 'group buying', 'groupware', 'gtc', 'guard interval', 'gui', 'handwriting recognition', 'haptics', 'hcm', 'hd voice', 'health information exchange', 'heijunka', 'heterogeneous architecture', 'hevs', 'hie', 'hierarchical database', 'hierarchical storage management', 'highlight color', 'hipaa', 'hit', 'hlr', 'hmds', 'home health monitoring', 'home server', 'hoshin', 'hot spot', 'hpc', 'hrms', 'hsm', 'html', 'html5', 'hub', 'huds', 'human augmentation', 'human capital management', 'human resource management system', 'humanitarian disaster relief', 'hvd', 'hybrid cloud computing', 'hybrid electric vehicles', 'hybrid modeling', 'hybrid mpls', 'hydrogen economy', 'hype cycle', 'hyperlink', 'hypertext', 'hypertext markup language', 'i/o', 'i/o bound', 'i/o channel', 'iaas', 'iae', 'iam', 'ian', 'ichat', 'ida', 'idea engine', 'idea management', 'idef', 'iden', 'idl', 'idm', 'idn', 'ids', 'ieee', 'iles', 'ilm', 'image recognition', 'imap', 'imei', 'immersive learning environments', 'imsi', 'independent software vendor', 'information', 'information architecture', 'information cards', 'information delivery', 'information dispersal algorithms', 'information engineering', 'information governance', 'information life cycle management', 'information management', 'information technology', 'infrastructure software', 'innovation management', 'input/output', 'inspect', 'instant messaging', 'integrated carrier', 'integrated delivery system', 'integrated development environment', 'integrated digital network', 'integrated document management', 'integrated receiver decoder', 'integrated services digital network', 'integrated software', 'integration', 'integration broker', 'integration brokerage', 'intellectual property', 'interactive visualization', 'interactive voice response', 'interface definition language', 'international mobile telecommunications advanced', 'international telecommunication union', 'internet', 'internet protocol', 'internet protocol address', 'internet protocol television', 'internet protocol version 6', 'internet radio', 'internet service provider', 'internet telephony', 'internet tv', 'internet wan', 'interoperability', 'intranet', 'introspection', 'ip address', 'ip datacasting', 'ip multimedia subsystem', 'ip telephony', 'ipa', 'iphone', 'iptv', 'ipv6', 'ird', 'irda', 'irr', 'isdn', 'iso', 'iso 9000', 'isp', 'isv', 'it decision support', 'it governance', 'it infrastructure', 'it management', 'it operations', 'it outsourcing', 'it risk', 'it services', 'it strategy', 'itg', 'itil', 'itu', 'ivr', 'java', 'java applet', 'java ee', 'java me', 'java platform', 'java platform', 'java servlet', 'javascript', 'jidoka', 'jit', 'job scheduling', 'jukebox', 'kaizen', 'kanban', 'kano model', 'kcc', 'kernel', 'key performance indicator', 'km strategy', 'knowledge access', 'knowledge architect', 'knowledge assets', 'knowledge audit', 'knowledge base', 'knowledge capital', 'knowledge capture', 'knowledge community', 'knowledge management', 'knowledge map', 'knowledge organization', 'knowledge representation', 'knowledge sharing', 'knowledge use', 'knowledge users', 'knowledge workplace', 'kpi', 'labor management system', 'laboratory information management system', 'lan', 'laser', 'latency', 'lba', 'lbm', 'lbs', 'lcd', 'lcos', 'lcr', 'ldap', 'lead management', 'lean', 'lean enterprise', 'lean thinking', 'learning stack', 'led', 'leo', 'level', 'library', 'license', 'life span', 'lifepo4 batteries', 'lightweight directory access protocol', 'lims', 'line', 'line balancing', 'line driver', 'line loading', 'linux', 'liquefied natural gas', 'liquid cooling', 'liquid crystal display', 'liquid submersion cooling', 'lisp', 'lithium iron phosphate', 'lng', 'load balancing', 'load forecasting', 'local multipoint distribution service', 'local positioning system', 'logistics network planning', 'loopback', 'low earth orbit', 'low frequency', 'lrc', 'lurker', 'm2m communications', 'machine learning', 'magnetometer', 'mainframe', 'mainframe', 'managed file transfer', 'managed object', 'managed print services', 'managed security service provider', 'management consulting', 'management information base', 'manufacturer', 'manufacturing execution system', 'manufacturing planning', 'manufacturing scheduling', 'mapi', 'mapreduce', 'mashup', 'mashups', 'mass collaboration', 'master data management', 'materials management', 'mcm', 'mda', 'mdm', 'media access control', 'media gateways', 'media objects', 'media tablet', 'medical management', 'megaportal', 'memory overcommit', 'memristor', 'mesh network', 'message authentication', 'message authentication code', 'message broker', 'message feedback', 'message format', 'message passing', 'message queuing', 'message switching', 'messaging', 'metadata', 'metadirectory', 'metalanguage', 'metrication', 'mft', 'mhl', 'micro fuel cells', 'microblogging', 'microcode', 'microfilm', 'microgrid', 'microgrids', 'microprocessor', 'middleware', 'minimum cell rate', 'mips', 'mms', 'mobile', 'mobile advertising', 'mobile application stores', 'mobile browsers', 'mobile device management', 'mobile drm', 'mobile earth station', 'mobile im', 'mobile ip', 'mobile middleware', 'mobile network', 'mobile payment', 'mobile pc', 'mobile portal', 'mobile satellite service', 'mobile satellite services', 'mobile social networks', 'mobile tv', 'mobile virtual network operator', 'mobile web applications', 'mobile widgets', 'modulation', 'molecular transistors', 'money clouds', 'mooc', 'mood recognition', 'motion jpeg', 'moves', 'mpeg', 'mpm', 'mps', 'mso', 'msp', 'msps', 'mss', 'mssp', 'multicarrier code division multiple access', 'multichannel multipoint distribution service', 'multicore processor', 'multidimensional database management system', 'multimedia', 'multimedia markup language', 'multimedia messaging service', 'multimode fiber', 'multiple input/multiple output', 'multiple instruction', 'multiplexing', 'multipoint', 'multitenancy', 'multithreading', 'multitouch', 'mvno', 'mvoip', 'nac', 'nagara', 'nanotube', 'nap', 'narrowband channels', 'nas', 'ndc', 'ndf', 'near field communication', 'nemawashi', 'netbooks', 'network', 'network access control', 'network appliance', 'network computer', 'network computing', 'network database', 'network function virtualization', 'network intelligence', 'network interface card', 'network inventory', 'network management', 'network management center', 'network operating system', 'network outsourcing', 'network redundancy', 'network security', 'network sharing', 'network topology', 'network virtualization', 'networked data center', 'nfc', 'nfv', 'nlp', 'nmos', 'noc', 'node b', 'nomadic wireless', 'notebook', 'oasis', 'object class', 'object data model', 'object instance', 'object request broker', 'object role modeling', 'occam process', 'odm', 'odp', 'oled', 'olfactory interfaces', 'onboarding', 'oneapi', 'online transaction processing', 'open architecture', 'open data', 'open source', 'openstack', 'operating system', 'operational data store', 'operational resilience', 'operations services', 'operations support system', 'optical transport', 'optimization routines', 'option analysis', 'order management', 'organizational', 'original equipment manufacturer', 'osi management', 'outage management system', 'overlay', 'p2p', 'paas', 'packet control unit', 'packet switching', 'pacs', 'pad', 'page', 'pal', 'pam', 'paper size', 'parabolic trough', 'parallel network file system', 'parallel processing', 'partitioning code', 'partner relationship management', 'pas', 'pat', 'patient care management', 'pbx', 'pcm', 'pco', 'pcs', 'pcu', 'pda', 'pdc', 'pde', 'pdm', 'peak traffic', 'penetration rate', 'penetration testing', 'perfection', 'performance management', 'personal cloud', 'personal communications services', 'personal digital assistant', 'personal digital cellular', 'personal handyphone system', 'personal health record', 'personal information manager', 'phase change memory', 'phasor measurement units', 'pho', 'phr', 'phs', 'pico projector', 'picocell', 'pim', 'pims', 'pki', 'plc', 'plm', 'pmo', 'pmps', 'pmus', 'pnfs', 'podcast', 'podcasting', 'polymer memory', 'pop', 'portable wireless', 'portal', 'portfolio management', 'porting', 'pox', 'ppm', 'predictive analytics', 'predictive modeling', 'predictive support', 'prescriptive analytics', 'presence', 'privacy management tools', 'private branch exchange', 'private cloud computing', 'private exchange', 'private key', 'prm', 'process control', 'process management', 'process manufacturing', 'process templates', 'processing', 'processor emulation', 'procurement applications', 'procurement network', 'product catalog', 'product configurators', 'product cost management', 'product data management', 'product life cycle management', 'production devices', 'production information management system', 'programmable logic controller', 'programming language', 'project management', 'project management office', 'proprietary software', 'protocol', 'protocol conversion', 'protocol stack', 'proxy agent', 'proxy servers', 'psa', 'psk', 'ptr', 'public cloud computing', 'public key', 'publish/subscribe architecture', 'pull', 'pulse amplitude modulation', 'pulse code modulation', 'pulse width modulation', 'push technology', 'pwm', 'pwn', 'qam', 'qos', 'qr codes', 'quadrature amplitude modulation', 'quality assurance', 'quantum computer', 'quick response codes', 'raad', 'rack', 'rack density', 'rack unit', 'rad', 'radio network controller', 'raid', 'rapid application development', 'ras', 'rasp', 'rcs', 'rdbms', 'rdm', 'real time', 'receiver sensitivity', 'reconfigurable optical add/drop multiplexers', 'records management', 'redaction', 'reduced instruction set computer', 'redundancy', 'regulatory', 'regulatory compliance', 'relational database management system', 'relational dbms', 'relational online analytical processing', 'relationship manager', 'reliability', 'remote diagnostics', 'repository', 'response time', 'retail sales', 'reuse', 'revenue', 'revenue assurance', 'rf power amplifiers', 'rfid', 'rfid reader', 'rfid tags', 'ria', 'rich communication suite', 'rich internet application', 'risc', 'rnc', 'roadms', 'roaming', 'roc', 'roe', 'roi', 'rolap', 'rsa', 'rtt', 'rvm', 'saas', 'sac', 'sae', 'sales force automation', 'san', 'sast', 'satellite communications', 'satellite computer', 'satellite dish', 'satellite navigation systems', 'satellite phone', 'satphone', 'sax', 'sbm', 'scada', 'scalability', 'scanner', 'scattering', 'sccm', 'sce', 'scm', 'scp', 'scpc', 'scr', 'scrambler', 'screen sharing', 'screening', 'sdh/sonet', 'sdk', 'sdma', 'sdn', 'sdr', 'sdsl', 'search engine', 'secondary channel', 'secondhand market', 'secret key', 'secure sockets layer', 'selector', 'semantic data model', 'semantic nets', 'sensei', 'serial interface', 'serial transmission', 'server appliance', 'service bureau', 'service catalog', 'service desk', 'service inventory', 'service levels', 'service pack', 'servlet', 'session border controllers', 'session initiation protocol', 'sfa', 'sfc', 'sfdr', 'sfm', 'sgsn', 'shadow it', 'shim', 'shipments', 'shojinka', 'shop floor control', 'short message service', 'sideband', 'siem', 'signature', 'signature verification', 'sim application toolkit', 'sim card', 'sim toolkit', 'simd', 'simm', 'simple mail transfer protocol', 'simple network management protocol', 'simple object access protocol', 'simple workflow access protocol', 'simplex', 'simulation', 'single instruction', 'sink', 'sio', 'sip', 'site', 'six sigma', 'skill mining', 'skinput', 'sku', 'sla', 'sled', 'slm', 'slo', 'smart antenna', 'smart card', 'smart grid', 'smart terminal', 'smartphone', 'smb', 'sme', 'smed', 'smp', 'smr', 'sms', 'smtp', 'sna', 'sniffer', 'snmp', 'snomed', 'soa', 'soap', 'soc', 'social analytics', 'social bpm', 'social computing', 'social content', 'social crm', 'social feedback', 'social gaming', 'social media', 'social media tools', 'social network analysis', 'social networking', 'social networking sites', 'social profile', 'social profiles', 'social publishing', 'social search', 'social software', 'social systems', 'social tagging', 'social technologies', 'social web', 'sockets', 'software asset management', 'software configuration management', 'software defined networking', 'software development', 'software development kit', 'software license management', 'software maintenance', 'soho', 'solution', 'solution architecture', 'solution portfolio', 'sonet', 'source encoding', 'source routing', 'source traffic descriptor', 'sow', 'space segment', 'spam', 'spamdexing', 'spatial division multiple access', 'spc', 'spec', 'specification', 'spectrum', 'speech recognition', 'spf', 'spider', 'spoofing', 'spp', 'spread spectrum', 'sql', 'sra', 'srm', 'ssl', 'ssn', 'sso', 'ssp', 'sspc', 'standard', 'standards', 'static application security testing', 'statistical multiplexing', 'statistical process control', 'stdm', 'step', 'stickiness', 'stm', 'storage appliance', 'storage management software', 'storage resource management', 'storage service provider', 'strategy', 'streaming', 'street', 'structural change', 'structure', 'structured query language', 'subjective', 'subnet', 'subscriber', 'superserver', 'supply chain execution', 'supply chain management', 'supply chain planning', 'surface computers', 'sustainability management', 'sustainable cell rate', 'svga', 'swap', 'swift', 'switch', 'switched line', 'switched network', 'switching', 'switching center', 'switchover', 'symbian', 'symmetric digital subscriber line', 'symmetric multiprocessing', 'synchronization', 'synchronous', 'synchronous communications', 'synchronous network', 'synchronous optical network', 'synchronous transfer mode', 'synchronous transmission', 'syncml', 'syntax', 'system architecture evolution', 'system integration', 'system integrator', 'system management', 'table driven', 'tacit knowledge', 'tacs', 'tag management', 'takt time', 'tco', 'tcp', 'tcp/ip', 'tdm', 'tdma', 'tdr', 'telco', 'telecommunications carrier', 'telecommunications equipment', 'telecommunications services', 'teledensity', 'telematics', 'term', 'terminal', 'terminal emulation', 'text analytics', 'text mining', 'text retrieval', 'thermal printing', 'thermal transfer', 'thin client', 'throughput', 'time division multiple access', 'time division multiplexing', 'time domain reflectometry', 'time out', 'tlm', 'tlp', 'tls', 'tmg', 'tms', 'tokenization', 'total revenue', 'touchpoint', 'transceiver', 'transcoding server', 'transducer', 'transfer rate', 'translator', 'transmedia', 'transmission control protocol', 'transmission control protocol/internet protocol', 'transparency', 'transponder', 'transport layer security', 'transportation management system', 'trouble ticket', 'tsvs', 'tuning', 'tunneling', 'ucc', 'ultrabook', 'ultracapacitors', 'unified communications', 'unified threat management', 'uniform resource locator', 'url', 'user authentication technologies', 'user provisioning', 'utm', 'value stream', 'value stream mapping', 'van', 'var', 'variable bit rate', 'vas', 'vba', 'vbr', 'vdi', 'vdsl', 'vendor', 'vendor management', 'vendor revenue', 'video telepresence', 'videoconferencing', 'virtual assistant', 'virtual channel', 'virtual desktop infrastructure', 'virtual lan', 'virtual machine', 'virtual network operator', 'virtual private network', 'virtual reality', 'virtual reality modeling language', 'virtual switch', 'virtualization', 'virtualization software', 'visitor location register', 'visual basic', 'visual studio', 'visualization', 'vlan', 'vlr', 'vlsi', 'voc', 'vod', 'voice browser', 'voice encryption', 'voice mail', 'voice portal', 'voice response system', 'voice response unit', 'voice switching', 'voicexml', 'voip', 'volumetric displays', 'vowlan', 'vpn', 'vrm', 'vrml', 'vrs', 'vru', 'vsat', 'vsf', 'vsm', 'vtl', 'w3c', 'wafer', 'wag', 'wam', 'wap', 'wap forum', 'warehouse simulation', 'wasp', 'waste', 'wave power', 'wbs', 'wcdma', 'wcm', 'wdp', 'wearable computer', 'web', 'web analytics', 'web atm', 'web books', 'web content management', 'web crawler', 'web hosting', 'web phone', 'web server', 'web services', 'web services software', 'web tv', 'web widgets', 'website', 'wep', 'wfm', 'wibro', 'wideband code division multiple access', 'wigig', 'wiki', 'wim', 'wimax', 'windowing', 'windows ce', 'windows client', 'windows live messenger', 'wips', 'wired equivalent privacy', 'wireless application protocol', 'wireless application service provider', 'wireless broadband', 'wireless data communication', 'wireless fidelity', 'wireless hd', 'wireless power', 'wlan', 'wll', 'woa', 'wocs', 'work management', 'workflow management', 'workforce analytics', 'workforce management', 'workstations', 'world wide web consortium', 'wpa', 'wpa2', 'x windows', 'xbrl', 'xml', 'xsl', 'xslt', 'zigbee', 'zle']

In [130]:
stop_words = """a,z,about,above,across,after,again,against,all,almost,alone,along,already,also,although,always,among,an,and,another,any,anybody,anyone,anything,anywhere,are,area,areas,around,as,ask,asked,asking,asks,at,away,b,back,backed,backing,backs,be,became,because,become,becomes,been,before,began,behind,being,beings,best,better,between,big,both,but,by,came,can,cannot,case,cases,certain,certainly,clear,clearly,come,could,d,did,differ,different,differently,do,does,done,down,down,downed,downing,downs,during,e,each,early,either,end,ended,ending,ends,enough,even,evenly,ever,every,everybody,everyone,everything,everywhere,f,face,faces,fact,facts,far,felt,few,find,finds,first,for,four,from,full,fully,further,furthered,furthering,furthers,g,gave,general,generally,get,gets,give,given,gives,go,going,good,goods,got,great,greater,greatest,group,grouped,grouping,groups,h,had,has,have,having,he,her,here,herself,high,high,high,higher,highest,him,himself,his,how,however,i,if,important,in,interest,interested,interesting,interests,into,is,it,its,itself,j,just,k,keep,keeps,kind,knew,know,known,knows,l,large,largely,last,later,latest,least,less,let,lets,like,likely,long,longer,longest,m,made,make,making,man,many,may,me,member,members,men,might,more,most,mostly,mr,mrs,much,must,my,myself,n,necessary,need,needed,needing,needs,never,new,new,newer,newest,next,no,nobody,non,noone,not,nothing,now,nowhere,number,numbers,o,of,off,often,old,older,oldest,on,once,one,only,open,opened,opening,opens,or,order,ordered,ordering,orders,other,others,our,out,over,p,part,parted,parting,parts,per,perhaps,place,places,point,pointed,pointing,points,possible,present,presented,presenting,presents,problem,problems,put,puts,q,quite,r,rather,really,right,right,room,rooms,s,said,same,saw,say,says,second,seconds,see,seem,seemed,seeming,seems,sees,several,shall,she,should,show,showed,showing,shows,side,sides,since,small,smaller,smallest,so,some,somebody,someone,something,somewhere,state,states,still,still,such,sure,t,take,taken,than,that,the,their,them,then,there,therefore,these,they,thing,things,think,thinks,this,those,though,thought,thoughts,three,through,thus,to,today,together,too,took,toward,turn,turned,turning,turns,two,u,under,until,up,upon,us,use,used,uses,v,very,w,want,wanted,wanting,wants,was,way,ways,we,well,wells,went,were,what,when,where,whether,which,while,who,whole,whose,why,will,with,within,without,work,worked,working,works,would,x,y,year,years,yet,you,young,younger,youngest,your,yours,z"""

## DB 불러오기 

In [152]:
db = pymysql.connect(host='218.150.181.120',port=33060,user='root',
                     passwd='1234#link',db='papercrawler',charset='utf8', use_unicode=True)
cursor = db.cursor()
sql1 = """select abstractPaper from papercrawler.abstractManuscript where manuscriptId in 
(SELECT id FROM papercrawler.manuscript where year >= 1946 and year <= 1955)"""

cursor.execute(sql1)

result=cursor.fetchall()

In [ ]:
db = pymysql.connect(host='218.150.181.120',port=33060,user='root',
                     passwd='1234#link',db='papercrawler',charset='utf8', use_unicode=True)
cursor = db.cursor()
sql1 = """SELECT title FROM papercrawler.manuscript where year >= 1946 and year <= 1955"""

cursor.execute(sql1)

result2=cursor.fetchall()

In [132]:
text = str(result) #+ str(result2)
text = clr(text)
text = ' '.join([word for word in text.lower().split() if word not in stop_words])

In [167]:
textFile = sc.parallelize(text.split())

In [168]:
textFile.count()

3631

In [178]:
textFile.take(5)

['sommairel\\u2019auteur',
 'passe',
 'revue',
 'principaux',
 'd\\xe9veloppements']

In [170]:
linesWithSpark = textFile.filter(lambda line: "en" in line)

In [171]:
print linesWithSpark.count()

321


In [176]:
wordCounts = textFile.flatMap(lambda line: line.split()).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a+b)

In [179]:
tt = wordCounts.collect()

In [180]:
type(tt)

list

In [182]:
print tt[1]

('semantic', 1)
